In [1]:
import pandas as pd
import numpy as np


In [2]:
global ex_data_dir, in_data_dir, in_filter_dir
ex_data_dir = '../../data/processed/external/'
in_data_dir = '../../data/interim/internal/filtered_before_agg/'
in_filter_dir = '../../data/interim/internal/filter_before_agg/'


In [3]:
pref_list = [
    "北海道",
    "青森県",
    "岩手県",
    "宮城県",
    "秋田県",
    "山形県",
    "福島県",
    "茨城県",
    "栃木県",
    "群馬県",
    "埼玉県",
    "千葉県",
    "東京都",
    "神奈川県",
    "新潟県",
    "富山県",
    "石川県",
    "福井県",
    "山梨県",
    "長野県",
    "岐阜県",
    "静岡県",
    "愛知県",
    "三重県",
    "滋賀県",
    "京都府",
    "大阪府",
    "兵庫県",
    "奈良県",
    "和歌山県",
    "鳥取県",
    "島根県",
    "岡山県",
    "広島県",
    "山口県",
    "徳島県",
    "香川県",
    "愛媛県",
    "高知県",
    "福岡県",
    "佐賀県",
    "長崎県",
    "熊本県",
    "大分県",
    "宮崎県",
    "鹿児島県",
    "沖縄県",
]

In [5]:
df = pd.read_csv(in_data_dir + 'addedclassification.csv', 
                encoding='utf-8', 
                sep=',', 
                dtype=str)
df

city_df = pd.read_csv(
    in_filter_dir + "jp_address_flagged.csv", encoding="utf-8", sep=",", dtype=str
)
city_df

,city,prefecture
0,大韓民国,NaN
1,福岡市,福岡県
2,神戸市,兵庫県
3,大阪市,大阪府
4,京都市,京都府
...,...,...
645,イギリス領ケイマン諸島,NaN
646,志太郡大井川町,静岡県
647,ロシア連邦,NaN
648,オランダ国,NaN


In [ ]:
# df[~df['prefecture'].isin(pref_list)]['right_person_addr'].unique()

In [6]:
japan_df = pd.merge(df, city_df, left_on='right_person_addr', right_on='city', 
         how='left').drop(columns=['city']).copy()
for pref in pref_list:
    japan_df['prefecture'] = np.where(japan_df['right_person_addr'].str.startswith(pref), 
                                pref, 
                                japan_df['prefecture'])
japan_df = japan_df.dropna(subset=['prefecture'])\
                    .drop(columns=['right_person_addr'])\
                    .rename(columns={'prefecture': 'right_person_addr'})
japan_df


,reg_num,ipc,right_person_name,app_year,app_nendo,reg_year,reg_nendo,schmoch35,right_person_addr
0,5684492,G10H 3/18,ＤＲＣ合同会社,2010,2010,2015,2014,34,東京都
1,5684512,B62D 57/02,株式会社ＩＨＩエアロスペース,2010,2010,2015,2014,32,東京都
2,5684598,A47C 7/62,株式会社オカムラ,2011,2010,2015,2014,33,神奈川県
3,5684620,H01M 4/62,三井化学株式会社,2011,2010,2015,2014,1,東京都
4,5684633,G06F 12/02,日本電信電話株式会社,2011,2011,2015,2014,6,東京都
...,...,...,...,...,...,...,...,...,...
6038763,7143985,C12Q 1/28,兵庫県,2018,2017,2022,2022,15,兵庫県
6038764,7143996,A61L 15/22,滋賀県,2018,2017,2022,2022,13,滋賀県
6038765,7143997,D06L 4/24,岡山県,2018,2017,2022,2022,23,岡山県
6038766,7145706,G06F 16/735,日本放送協会,2018,2018,2022,2022,6,東京都


In [7]:
japan_df['right_person_addr'] = japan_df['right_person_addr'].str.replace('省略', '')
japan_df['right_person_addr'] = pd.Categorical(japan_df['right_person_addr'], 
                                               categories=pref_list+[''], 
                                               ordered=True)
japan_df = japan_df.sort_values(by=['app_year', 'right_person_name', 'right_person_addr'], 
                                ascending=True)
japan_df['right_person_addr'] = japan_df['right_person_addr'].replace('', np.nan).ffill()
japan_df


,reg_num,ipc,right_person_name,app_year,app_nendo,reg_year,reg_nendo,schmoch35,right_person_addr
2640459,641649,G03G 5/14,コニカミノルタ株式会社,1960,1960,1972,1972,9,東京都
2608047,580357,H04N 5/783,ソニーグループ株式会社,1960,1960,1970,1970,2,東京都
2608046,580357,H04N 5/783,ソニー株式会社,1960,1960,1970,1970,2,東京都
2649041,658692,D01D 5/253,トリニテイ・インベストメント株式会社,1960,1960,1972,1972,28,東京都
2649042,658693,D01D 5/253,トリニテイ・インベストメント株式会社,1960,1960,1972,1972,28,東京都
...,...,...,...,...,...,...,...,...,...
1410797,7325879,H04M 11/00,ａｍｐｔａｌｋ株式会社,2023,2023,2023,2023,3,東京都
1523576,7344612,G10L 15/10,ａｍｐｔａｌｋ株式会社,2023,2023,2023,2023,6,東京都
1349660,7315191,G06Q 10/00,ｂｏｏｏｓｔｔｅｃｈｎｏｌｏｇｉｅｓ株式会社,2023,2022,2023,2023,7,東京都
1452345,7335032,G01F 19/00,ｉＸＩＴ株式会社,2023,2023,2023,2023,10,東京都


In [8]:
in_data_dir

'../../data/interim/internal/filtered_before_agg/'

In [9]:
japan_df.to_csv(in_data_dir + 'japan.csv', 
                encoding='utf-8', 
                sep=',', 
                index=False)